In [19]:
import pygame
import numpy as np
import heapq
import random

# pygame Initialization and set up display
pygame.init()
SCREEN_WIDTH, SCREEN_HEIGHT = 750, 750
ROWS, COLS = 40, 40  # Grid size
TILE_SIZE = SCREEN_WIDTH // COLS # Size of each grid tile
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Cleaning Robot Simulation ")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (128, 128, 128)
BROWN = (165, 42, 42)
BEIGE = (165, 142, 102) 


# Load images
ROBOT_IMAGE = pygame.image.load('images/robotC.png')  # Path to robot image
OBSTACLE_IMAGE = pygame.image.load('images/obstacleC.png')  # Path toobstacle image
DUST_IMAGE = pygame.image.load('images/dustC.png')  # Path to  dust image
DUST_IMAGE = pygame.transform.scale(DUST_IMAGE, (TILE_SIZE, TILE_SIZE))

# Load furniture images
DESK_IMAGE = pygame.image.load('images/dinnerC.png')
DESK_IMAGE = pygame.transform.scale(DESK_IMAGE, (TILE_SIZE * 7, TILE_SIZE * 6))
DESK1_IMAGE = pygame.image.load('images/meetingC.png')
DESK1_IMAGE = pygame.transform.scale(DESK1_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 8))
DESK2_IMAGE = pygame.image.load('images/deskC.png')
DESK2_IMAGE = pygame.transform.scale(DESK2_IMAGE, (TILE_SIZE * 7, TILE_SIZE * 7))
DESK3_IMAGE = pygame.image.load('images/deskC1.png')
DESK3_IMAGE = pygame.transform.scale(DESK3_IMAGE, (TILE_SIZE * 4, TILE_SIZE * 4))
SOFA_IMAGE = pygame.image.load('images/sofaC.png')
SOFA_IMAGE = pygame.transform.scale(SOFA_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 8))
DESK4_IMAGE = pygame.image.load('images/deskC2.png')
DESK4_IMAGE = pygame.transform.scale(DESK4_IMAGE, (TILE_SIZE * 4, TILE_SIZE * 4))
DESK5_IMAGE = pygame.image.load('images/deskC3.png')
DESK5_IMAGE = pygame.transform.scale(DESK5_IMAGE, (TILE_SIZE * 4, TILE_SIZE * 5))
DESK6_IMAGE = pygame.image.load('images/deskC4.png')
DESK6_IMAGE = pygame.transform.scale(DESK6_IMAGE, (TILE_SIZE * 6, TILE_SIZE * 7))
FURN_IMAGE = pygame.image.load('images/furnC.png')
FURN_IMAGE = pygame.transform.scale(FURN_IMAGE, (TILE_SIZE * 7, TILE_SIZE * 2))
FURN1_IMAGE = pygame.image.load('images/furnC1.png')
FURN1_IMAGE = pygame.transform.scale(FURN1_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 2))
FURN2_IMAGE = pygame.image.load('images/furnC2.png')
FURN2_IMAGE = pygame.transform.scale(FURN2_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 2))
FURN3_IMAGE = pygame.image.load('images/furnC3.png')
FURN3_IMAGE = pygame.transform.scale(FURN3_IMAGE, (TILE_SIZE * 2, TILE_SIZE * 8))


# Default positions for furniture items (using -1 to indicate no placement yet)
desk_x = desk_y = desk1_x = desk1_y = desk2_x = desk2_y = desk3_x = desk3_y = -1
sofa_x = sofa_y = desk4_x = desk4_y = desk5_x = desk5_y = desk6_x = desk6_y = furn_x = furn_y = furn1_x = furn1_y = furn2_x = furn2_y = furn2_x = furn2_y = -1

# Define initial positions list for all desks and sofas
desk_positions = [(0, 0)] * 6 
sofa_position = (-1, -1) 


# Adjust robot size and scale
ROBOT_IMAGE_SIZE = TILE_SIZE  # Change size  to make the robot bigger
ROBOT_IMAGE = pygame.transform.scale(ROBOT_IMAGE, (ROBOT_IMAGE_SIZE, ROBOT_IMAGE_SIZE))

OBSTACLE_IMAGE = pygame.transform.scale(OBSTACLE_IMAGE, (TILE_SIZE, TILE_SIZE))

# Room grid, dust, robots, and obstacles
grid = [[0 for _ in range(COLS)] for _ in range(ROWS)]
dust_positions = []
robots = []
moving_obstacles = []


# to store the positions of all robots
robot_positions = {}

# Function to randomly place dust particles on the grid
def spawn_dust():
    for _ in range(150):  # Number of dust particles
        x, y = random.randint(0, ROWS - 1), random.randint(0, COLS - 1)
        if grid[x][y] == 0:  # Ensure it's an empty spot
            dust_positions.append((x, y))

# Robot class with methods for movement, cleaning, and drawing
class Robot:
    def __init__(self, x, y, id):
        self.x, self.y = x, y
        self.id = id  # Unique ID for each robot
        self.path = []
        robot_positions[self.id] = (self.x, self.y)  # Initialize position

    def move_towards(self, target):
        if self.path:
            next_position = self.path[0]
            
            # Check if the next position is already occupied by another robot
            if not will_collide(next_position) and not is_occupied_by_robot(next_position, self.id):
                # Update robot's position
                self.x, self.y = self.path.pop(0)
                robot_positions[self.id] = (self.x, self.y)  # Update position in dictionary
                update_map(self)  # Update the map each time the robot moves
            else:
                # Recalculate path if a collision is predicted
                self.path = a_star_search((self.x, self.y), target)
        else:
            self.path = a_star_search((self.x, self.y), target)

    def clean_dust(self):
        # Remove dust if robot is on a dust position
        global dust_positions
        dust_positions = [d for d in dust_positions if d != (self.x, self.y)]

    def draw(self):
        # robot's image at its current position
        screen.blit(ROBOT_IMAGE, (self.y * TILE_SIZE, self.x * TILE_SIZE))

    def update_path(self):
        # Update path if obstacle is detected in current path
        if self.path and (self.x, self.y) != self.path[0]:
            target = (self.path[0])
            if is_obstacle_in_path(target):
                self.path = a_star_search((self.x, self.y), target)
                if not self.path:
                    self.path = rrt_pathfinding((self.x, self.y), target)


# function to check if a position is occupied by another robot
def is_occupied_by_robot(position, robot_id):
    for other_robot_id, other_position in robot_positions.items():
        if other_robot_id != robot_id and other_position == position:
            return True
    return False

def will_collide(next_position):
    """
    Check if any moving obstacle is currently or will be at the next_position.
    """
    for obstacle in moving_obstacles:
        future_x, future_y = obstacle.x + obstacle.direction[0], obstacle.y + obstacle.direction[1]
        if (obstacle.x, obstacle.y) == next_position or (future_x, future_y) == next_position:
            return True
    return False


# Class for moving obstacles in the grid
class MovingObstacle:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])

    def is_near_robot(self, robots):
        for robot in robots:
            if abs(robot.x - self.x) <= 1 and abs(robot.y - self.y) <= 1:
                return True
        return False

    def move(self, robots):
        if random.random() < 0.1:
            self.direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
        
        # Calculate new position
        new_x, new_y = self.x + self.direction[0], self.y + self.direction[1]
        
        # Check if the new position is within bounds, not a wall, and away from robots
        if (0 <= new_x < ROWS and 0 <= new_y < COLS and 
            grid[new_x][new_y] != 2 and not self.is_near_robot(robots)):
            self.x, self.y = new_x, new_y
        else:
            # Choose a new direction if the current one would lead to a collision
            self.direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])

    def draw(self):
        screen.blit(OBSTACLE_IMAGE, (self.y * TILE_SIZE, self.x * TILE_SIZE))

# Function to detect if a target position has a moving obstacle        
def is_obstacle_in_path(target):
    return any((obstacle.x, obstacle.y) == target for obstacle in moving_obstacles)
    
#Create a grid that marks moving obstacles as walls.
def get_dynamic_grid():
    dynamic_grid = [row[:] for row in grid] 
    for obstacle in moving_obstacles:
        dynamic_grid[obstacle.x][obstacle.y] = 2  
    return dynamic_grid

# A* Pathfinding Algorithm
def a_star_search(start, goal):
    dynamic_grid = get_dynamic_grid()  # Get the updated grid for moving obstacles
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(current, dynamic_grid):
            tentative_g_score = g_score[current] + 1
            
            if tentative_g_score < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)

                if f_score[neighbor] == tentative_g_score:  # No heuristic effect
                    heapq.heappush(open_set, (tentative_g_score, neighbor))
                else:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []

# RRT Pathfinding Algorithm
def rrt_pathfinding(start, goal, max_iter=1000):
    dynamic_grid = get_dynamic_grid()  # Get the updated grid for moving obstacles
    tree = {start: None}
    for _ in range(max_iter):
        rand_point = (random.randint(0, ROWS - 1), random.randint(0, COLS - 1))
        nearest = min(tree.keys(), key=lambda p: heuristic(p, rand_point))
        
        new_point = move_towards(nearest, rand_point)
        if is_valid_point(new_point, dynamic_grid):
            tree[new_point] = nearest
            
            if heuristic(new_point, goal) < 1: 
                return reconstruct_rrt_path(tree, new_point, start)

    return []



def is_valid_point(point, dynamic_grid):
    return 0 <= point[0] < ROWS and 0 <= point[1] < COLS and dynamic_grid[int(point[0])][int(point[1])] == 0
def reconstruct_rrt_path(tree, goal, start):
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = tree[current]
    path.reverse()
    return path

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(pos, dynamic_grid):
    neighbors = [(pos[0] + dx, pos[1] + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
    return [(r, c) for r, c in neighbors if 0 <= r < ROWS and 0 <= c < COLS and dynamic_grid[r][c] != 2]

# Room setup for grid, walls, and openings
def create_rooms():
    global desk_x, desk_y, desk1_x, desk1_y, desk2_x, desk2_y, desk3_x, desk3_y, sofa_x, sofa_y, desk4_x, desk4_y, desk5_x, desk5_y, desk6_x, desk6_y, furn_x, furn_y, furn1_x, furn1_y, furn2_x, furn2_y, furn3_x, furn3_y
    mid_col = COLS // 2
    mid_row = ROWS // 2

    # Create walls dividing the rooms
    for row in range(ROWS):
        if not (mid_row - 16 <= row <= mid_row + 2):
            grid[row][mid_col] = 2  # Add a wall in the middle column
            
    # door opening in the middle wall
    grid[ROWS - 5][mid_col] = 0  # Open door
    grid[ROWS - 4][mid_col] = 0  # Open door

    for col in range(COLS):
        if not (1 <= col <= 2 or (COLS // 2 - 1 <= col <= COLS // 2)):
            grid[mid_row][col] = 2  # Add a wall across the middle row

    small_room_width = COLS // 3
    small_room_height = ROWS // 3

    for row in range(small_room_height):
        if row == 1 or row == 2:
            continue
        grid[row][small_room_width] = 2  # Left small room
        grid[row][COLS - small_room_width - 1] = 2  # Right small room

    for col in range(small_room_width):
        if col == 1 or col == 2:
            continue
        grid[small_room_height][col] = 2  # Left small room
        grid[small_room_height][COLS - col - 1] = 2  # Right small room

    # additional walls and doors based on your configuration
    wall_rec1_x = mid_row - 13
    wall_rec1_y = mid_col -14
    wall_width1 = 7
    wall_height1 = 7

    if wall_rec1_x + wall_height1 <= ROWS and wall_rec1_y + wall_width1 <= COLS:
        for i in range(wall_height1):
            for j in range(wall_width1):
                grid[wall_rec1_x + i][wall_rec1_y + j] = 2

    wall_rec_x = mid_row + 7
    wall_rec_y = mid_col + 7
    wall_width = 8
    wall_height = 8

    if wall_rec_x + wall_height <= ROWS and wall_rec_y + wall_width <= COLS:
        for i in range(wall_height):
            for j in range(wall_width):
                grid[wall_rec_x + i][wall_rec_y + j] = 2

    wall_sq_x = mid_row - 14
    wall_sq_y = mid_col + 7

    for i in range(7):
        for j in range(7):
            grid[wall_sq_x + i][wall_sq_y + j] = 2

    wall_sq_x1 = mid_row - 20
    wall_sq_y1 = mid_col + 14

    for i in range(4):
        for j in range(4):
            grid[wall_sq_x1 + i][wall_sq_y1 + j] = 2

    wall_sofa_x1 = mid_row + 12
    wall_sofa_y1 = mid_col - 20

    for i in range(8):
        for j in range(8):
            grid[wall_sofa_x1 + i][wall_sofa_y1 + j] = 2

    wall_sq_X2 = mid_row + 1
    wall_sq_y2 = mid_col + 16
    sq_width = 4
    sq_height = 4

    if wall_sq_X2 + sq_height <= ROWS and wall_sq_y2 + sq_width <= COLS:
        for i in range(sq_height):
            for j in range(sq_width):
                grid[wall_sq_X2 + i][wall_sq_y2 + j] = 2

    wall_sq_x3 = mid_row - 14
    wall_sq_y3 = mid_col - 6

    for i in range(5):
        for j in range(4):
            grid[wall_sq_x3 + i][wall_sq_y3 + j] = 2

    
    wall_sq_x4 = mid_row + 8
    wall_sq_y4 = mid_col - 6

    for i in range(7):
        for j in range(6):
            grid[wall_sq_x4 + i][wall_sq_y4 + j] = 2

    
    wall_sq_X5 = mid_row + 1
    wall_sq_y5 = mid_col - 13
    sq_width5 = 7
    sq_height5 = 2

    if wall_sq_X5 + sq_height5 <= ROWS and wall_sq_y5 + sq_width5 <= COLS:
        for i in range(sq_height5):
            for j in range(sq_width5):
                grid[wall_sq_X5 + i][wall_sq_y5 + j] = 2

    wall_sq_X6 = mid_row + 18
    wall_sq_y6 = mid_col + 3
    sq_width6 = 8
    sq_height6 = 2

    if wall_sq_X6 + sq_height6 <= ROWS and wall_sq_y6 + sq_width6 <= COLS:
        for i in range(sq_height6):
            for j in range(sq_width6):
                grid[wall_sq_X6 + i][wall_sq_y6 + j] = 2

    wall_sq_X7 = mid_row - 20
    wall_sq_y7 = mid_col -  18
    sq_width7 = 8
    sq_height7 = 2

    if wall_sq_X7 + sq_height7 <= ROWS and wall_sq_y7 + sq_width7 <= COLS:
        for i in range(sq_height7):
            for j in range(sq_width7):
                grid[wall_sq_X7 + i][wall_sq_y7 + j] = 2

    wall_sq_X8 = mid_row - 20
    wall_sq_y8 = mid_col - 20
    sq_width8 = 2
    sq_height8 = 8

    if wall_sq_X8 + sq_height8 <= ROWS and wall_sq_y8 + sq_width8 <= COLS:
        for i in range(sq_height8):
            for j in range(sq_width8):
                grid[wall_sq_X8 + i][wall_sq_y8 + j] = 2




     # Place the desk image above the wall
    desk_x = wall_rec1_x  # Align desk with the wall (same x position)
    desk_y = wall_rec1_y  # Align desk with the wall (same y position)
    desk1_x = wall_rec_x  # Align desk with the wall (same x position)
    desk1_y = wall_rec_y  # Align desk with the wall (same y position)
    desk2_x = wall_sq_x  # Align desk with the wall (same x position)
    desk2_y = wall_sq_y  # Align desk with the wall (same y position)
    desk3_x = wall_sq_x1  # Align desk with the wall (same x position)
    desk3_y = wall_sq_y1 # Align desk with the wall (same y position)
    sofa_x = wall_sofa_x1
    sofa_y = wall_sofa_y1
    desk4_x = wall_sq_X2  # Align desk with the wall (same x position)
    desk4_y = wall_sq_y2 # Align desk with the wall (same y position)
    desk5_x = wall_sq_x3  # Align desk with the wall (same x position)
    desk5_y = wall_sq_y3 # Align desk with the wall (same y position)
    desk6_x = wall_sq_x4  # Align desk with the wall (same x position)
    desk6_y = wall_sq_y4 # Align desk with the wall (same y position)
    furn_x = wall_sq_X5
    furn_y = wall_sq_y5
    furn1_x = wall_sq_X6
    furn1_y = wall_sq_y6
    furn2_x = wall_sq_X7
    furn2_y = wall_sq_y7
    furn3_x = wall_sq_X8
    furn3_y = wall_sq_y8

   

# Main Loop
def main():
    global dust_positions, robots
    create_rooms()
    spawn_dust()

    # Create robots at different starting positions
    robots.append(Robot(5, 5, 1))
    robots.append(Robot(ROWS - 2, COLS - 3, 2))
    robots.append(Robot(ROWS // 2, COLS // 2 - 10, 3))
   # Create moving obstacles without grid marking
    for _ in range(5):
        moving_obstacles.append(MovingObstacle(random.randint(0, ROWS-1), random.randint(0, COLS-1)))

    clock = pygame.time.Clock()
    running = True

    while running:
        screen.fill(WHITE) # Fill screen background
        draw_grid()

        # Draw and move robots
        for robot in robots:
            # Randomly choose a dust position as target if any dust exists
            if dust_positions:
                target = random.choice(dust_positions)
                robot.move_towards(target)
            robot.clean_dust()
            robot.draw()

        # Draw and move obstacles
        for obstacle in moving_obstacles:
            obstacle.move(robots)
            obstacle.draw()

        # Draw dust as images
        for (x, y) in dust_positions:
            screen.blit(DUST_IMAGE, (y * TILE_SIZE, x * TILE_SIZE))

        

        # Draw desks and sofas
        screen.blit(DESK_IMAGE, (desk_y * TILE_SIZE, desk_x * TILE_SIZE))
        screen.blit(DESK1_IMAGE, (desk1_y * TILE_SIZE, desk1_x * TILE_SIZE))
        screen.blit(DESK2_IMAGE, (desk2_y * TILE_SIZE, desk2_x * TILE_SIZE))
        screen.blit(DESK3_IMAGE, (desk3_y * TILE_SIZE, desk3_x * TILE_SIZE))
        screen.blit(SOFA_IMAGE, (sofa_y * TILE_SIZE, sofa_x * TILE_SIZE))
        screen.blit(DESK4_IMAGE, (desk4_y * TILE_SIZE, desk4_x * TILE_SIZE))
        screen.blit(DESK5_IMAGE, (desk5_y * TILE_SIZE, desk5_x * TILE_SIZE))
        screen.blit(DESK6_IMAGE, (desk6_y * TILE_SIZE, desk6_x * TILE_SIZE))
        screen.blit(FURN_IMAGE, (furn_y * TILE_SIZE, furn_x * TILE_SIZE))
        screen.blit(FURN1_IMAGE, (furn1_y * TILE_SIZE, furn1_x * TILE_SIZE))
        screen.blit(FURN2_IMAGE, (furn2_y * TILE_SIZE, furn2_x * TILE_SIZE))
        screen.blit(FURN3_IMAGE, (furn3_y * TILE_SIZE, furn3_x * TILE_SIZE))
        pygame.display.flip()
        clock.tick(10)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.display.flip()  # Update the display
    clock.tick(60)  # Cap the frame rate

    pygame.quit()

def draw_grid():
    for row in range(ROWS):
        for col in range(COLS):
            color = BEIGE if grid[row][col] == 2 else WHITE  # Use white for all grid squares
            pygame.draw.rect(screen, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))
        
if __name__ == "__main__":
    main()
